In [1]:
import pandas as pd

posts_files = "99 text/data/reddit-selfposts/rspct_autos.tsv.gz"
posts_df = pd.read_csv(posts_files, sep="\t")

subred_file = "99 text/data/reddit-selfposts/subreddit_info.csv.gz"
subred_df = pd.read_csv(subred_file).set_index("subreddit")

df = posts_df.join(subred_df, on="subreddit")

## Standardizing Attribute Names

In [2]:
print(df.columns)

Index(['id', 'subreddit', 'title', 'selftext', 'category_1', 'category_2',
       'category_3', 'in_data', 'reason_for_exclusion'],
      dtype='object')


In [3]:
column_mapping = {
    'id': 'id',
    'subreddit': 'subreddit',
    'title': 'title',
    'selftext': 'text',
    'category_1': 'category',
    'category_2': 'subcategory',  
    'category_3': None, # no data
    'in_data': None, # not needed
    'reason_for_exclusion': None # not needed
}

# define remaining columns
columns = [c for c in column_mapping.keys() if column_mapping[c] is not None]

# select and rename those columns
df = df[columns].rename(columns=column_mapping)

df = df[df['category'] == 'autos']
df.sample(1).T

,7197
id,6nonl4
subreddit,Harley
title,Is anyone going to sturgis this year? Any advi...
text,"Hey y'all, so first off I'm new to this sub bu..."
category,autos
subcategory,harley davidson


### Saving and Loading DataFrame

In [4]:
df.to_pickle("reddit_dataframe.pkl")

In [5]:
import sqlite3

db_name = 'reddit-selfposts.db'
con = sqlite3.connect(db_name)
df.to_sql('posts', con, index = False, if_exists='replace')
con.close()

In [7]:
con = sqlite3.connect(db_name)
df = pd.read_sql('SELECT * FROM posts', con)
con.close()

## Identify Noise with Regular Expressions

In [8]:
text = """
After viewing the [PINKIEPOOL Trailer](https://www.youtu.be/watch?v=ieHRoHUg)
it got me thinking about the best match ups.
<lb>Here's my take:<lb><lb>[](/sp)[](/ppseesyou) Deadpool<lb>[](/sp)[](/ajsly)
Captain America<lb>"""

In [9]:
import re
RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

def impurity(text, min_len=10):
    """returns the share of suspicious characters in a text"""
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text)) / len(text)
    
print(impurity(text))

0.09009009009009009


In [12]:
# add a new column to data frame
df['impurity'] = df['text'].apply(impurity, min_len=10)

# get the top 3 records
df[['text', 'impurity']].sort_values('impurity', ascending=False).head(3)

,text,impurity
19682,Looking at buying a 335i with 39k miles and 11...,0.214716
12357,I'm looking to lease an a4 premium plus automa...,0.165099
2730,Breakdown below:<lb><lb>Elantra GT<lb><lb>2.0L...,0.139130


In [15]:
from packages.blueprints.exploration import count_words
count_words(df, column='text', preprocess=lambda t: re.findall(r'<[/w\]*'))

AttributeError: 'Series' object has no attribute 'progress_map'